<a href="https://colab.research.google.com/github/lucky096/llm_augmented_workflow/blob/main/SocialMediaAgents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain_groq==0.2.0 --progress-bar off
!pip install -qqq langgraph==0.2.22 --progress-bar off

## Agent Prompts

In [2]:
EDITOR_PROMPT = """
Rewrite for maximum social media engagement:

- Use attention-grabbing, concise language
- Inject personality and humor
- Optimize formatting (short paragraphs)
- Encourage interaction (questions, calls-to-action)
- Ensure perfect grammar and spelling
- Rewrite from first person perspective, when talking to an audience

Use only the information provided in the text. Think carefully
"""

In [3]:
TWITTER_PROMPT = """
Generate a hih-engagement tweet from the given text:
1. What problem does this solve?
2. Focus on the main technical points/features
3. Write a short, coherent paragraph (2-3 sentences max)
4. Use natural, conversational language
5. Optimize for virality: make it intriguing, relatable, or controversial
6. Exclude emojis and hashtags
"""

In [4]:
TWITTER_CRITIC_PROMPT = """
You are a Tweet Critic Agent. Your task is to analyze tweets and provide actionable feedback to make them more engaging, focus on:

1. Clarity: Is the message clear and easy to understand?
2. Hook: Does it grab attention in the first few words?
3. Brevity: Is it concise while maintaining impact?
4. Call-to-action: Does it encourage interaction or sharing?
5. Tone: Is it appropriate for the intended audience?
6. Storytelling: Does it evoke curiosity?
7. Remove hype: Does it promise more than it delivers?

Provide 2-3 specific suggestions to improve the tweet's engagement potential.
Do not suggest hashtags. Keep your feedback concise and actionable.

Your goal is to help the writer improve their social media writing skills and increase engegement with their posts.
"""

In [5]:
LINKEDIN_PROMPT = """
Write a compelling LinkedIn post from the given text. Structure it as follows:

1. Eye-catching headline (5-7 words)
2. Identify a key problem or challenge
3. Provide a bullet list of key benefits/features
4. Highlight a clear benefit or solution
5. Conclude with a thought-provoking question

Maintain a professional, informative tone. Avoid emojis and hashtags.
Keep the post concise (50-80 words) and relevant to the industry.
Focus on providing valuable insights or actionable takeaways that will resonate with professionals in the field.
"""

In [7]:
LINKEDIN_CRITIC_PROMPT = """
Your role is to analyze LinkedIn posts and provide actionable feedback to make them more engaging.
Focus on the following aspects:

1. Hook: Evaluate the opening line's ability to grab attention
2. Structure: Assess the post's flow and readability
3. Content value: Determine if the post provides useful information or insights.
4. Call-to-action: Check if there's a clear next step for readers.
5. Language: Suggest improvements in tone, style, and word choice
6. Visual elements: Recommend additions or changes to images, videos, or formatting.

For each aspect, provide:
- A brief assessment (1-2 sentences)
- A specific suggestion for improvement
- A concise example of the suggested change

Conclude with an overall recommendation for the most impactful change the author can make to increase engagement.
Your goal is to help the writer improve their social media writing skills and increase engagement with posts.
"""

In [9]:
from pydantic import BaseModel
from typing import List, Optional, TypedDict
#state
class Post(BaseModel):
  """A post written in different versions"""

  drafts: List[str]
  feedback: Optional[str]

class State(TypedDict):
  user_text: str
  target_audience: str
  edit_text: str
  tweet: Post
  linkedin_post: Post
  n_drafts: int

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile", api_key="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")

In [13]:
from langchain_core.messages import SystemMessage, HumanMessage

#nodes
def editor_node(state: State):
  prompt = f"""
  text:
  ```
  {state["user_text"]}
  ```
  """.strip()
  response = llm.invoke([SystemMessage(EDITOR_PROMPT), HumanMessage(prompt)])
  return {"edit_text": response.content}

In [14]:
def tweet_writer_node(state: State):

  post = state["tweet"]

  feedback_prompt = (
      ""
      if not post.feedback
      else f"""
            Tweet:
            ```
            {post.drafts[-1]}
            ```
            Use the feeback to improve it:
            ```
            {post.feedback}
            ```
            """.strip()
  )

  prompt = f"""
            text:
            ```
            {state["edit_text"]}
            ```

            {feedback_prompt}

            Target audience: {state["target_audience"]}

            Write only the text for the post
            """.strip()

  response = llm.invoke([SystemMessage(TWITTER_PROMPT), HumanMessage(prompt)])
  post.drafts.append(response.content)
  return {"tweet": post}

In [15]:
def linkedin_writer_node(state: State):

  post = state["linkedin_post"]

  feedback_prompt = (
      ""
      if not post.feedback
      else f"""
      LinkedIn post:
      ```
      {post.drafts[-1]}
      ```
      Use the feedback to improve it:
      ```
      {post.feedback}
      ```
      """.strip()
  )

  prompt = f"""
  text:
  ```
  {state["edit_text"]}
  ```
  {feedback_prompt}
  Target Audience: {state["target_audience"]}
  write only the text for the post
  """.strip()

  response = llm.invoke([SystemMessage(LINKEDIN_PROMPT), HumanMessage(prompt)])
  post.drafts.append(response.content)
  return {"linkedin_post": post}